# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,276.53,83,35,4.34,RO,1738455347
1,1,stanley,54.8680,-1.6985,276.16,84,0,4.12,GB,1738455348
2,2,jamestown,42.0970,-79.2353,261.95,85,0,1.54,US,1738455349
3,3,rio san juan,19.6439,-70.0763,296.94,87,6,4.03,DO,1738455350
4,4,whitehorse,60.7161,-135.0538,248.32,62,78,2.68,CA,1738455229


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
#gmaps.configure(api_key= geoapify_key)
#marker_locations = city_data_df[['Lat',"Lng"]].astype(float)
#humidity = city_data_df['Humidity'].tolist()

# Display the map
map_plot_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City"
)

#Display the map
map_plot_1

C:\Users\User\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df[city_data_df['Max Temp'] >=21]

# Drop any rows with null values
new_city_df.dropna()

# Display sample data
new_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,276.53,83,35,4.34,RO,1738455347
1,1,stanley,54.8680,-1.6985,276.16,84,0,4.12,GB,1738455348
2,2,jamestown,42.0970,-79.2353,261.95,85,0,1.54,US,1738455349
3,3,rio san juan,19.6439,-70.0763,296.94,87,6,4.03,DO,1738455350
4,4,whitehorse,60.7161,-135.0538,248.32,62,78,2.68,CA,1738455229


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,constantia,44.1833,28.6500,276.53,83,35,4.34,RO,1738455347,
1,1,stanley,54.8680,-1.6985,276.16,84,0,4.12,GB,1738455348,
2,2,jamestown,42.0970,-79.2353,261.95,85,0,1.54,US,1738455349,
3,3,rio san juan,19.6439,-70.0763,296.94,87,6,4.03,DO,1738455350,
4,4,whitehorse,60.7161,-135.0538,248.32,62,78,2.68,CA,1738455229,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
lat = (-90, 90)
lon = (-180, 180)
radius = 10000
limit = 20
filter = f"circle:{lon}, {lat},{radius}"
bias = f"proximity:{lon},{lat}"
categories = 'accomodation.hotel', 'accomodation', 'building.holiday_house'
params = {
    'categories': categories,
    'limit': limit,
    'filter': filter,
    'bias' : bias,
    'apiKey' :geoapify_key
}

    # YOUR CODE HERE


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat =row['Lat']
    lon=row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {lon},{lat},{radius}"
    params["bias"] = f"proximity: {lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_response = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError, TypeError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
constantia - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
rio san juan - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
xai-xai - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
newport - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
nuku'alofa - nearest hotel: No hotel found
college - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
fada - nearest hotel: No hotel found
ushu

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,constantia,44.1833,28.6500,276.53,83,35,4.34,RO,1738455347,No hotel found
1,1,stanley,54.8680,-1.6985,276.16,84,0,4.12,GB,1738455348,No hotel found
2,2,jamestown,42.0970,-79.2353,261.95,85,0,1.54,US,1738455349,No hotel found
3,3,rio san juan,19.6439,-70.0763,296.94,87,6,4.03,DO,1738455350,No hotel found
4,4,whitehorse,60.7161,-135.0538,248.32,62,78,2.68,CA,1738455229,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
549,549,antalaha,-14.9003,50.2788,298.12,92,93,3.09,MG,1738456010,No hotel found
550,550,pangoa,-12.1167,-73.0000,295.60,92,100,0.39,PE,1738456012,No hotel found
551,551,ouesso,1.6136,16.0517,296.58,58,97,1.51,CG,1738456013,No hotel found
552,552,tongchuanshi,35.0806,109.0897,272.04,72,100,4.25,CN,1738456014,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width=700,
    frame_height=500,
    tiles='OSM',
    size='Humidity',
    geo= True,
    scale =1,
    color = "City",
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)